In [7]:
%load_ext dotenv
%dotenv
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import boto3


s3_client = boto3.client(
    's3',
    endpoint_url='https://storage.yandexcloud.net',
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
)
BUCKET_NAME = 'tnn-hse-medtech'
DATASET_DIR = 'datasets/'


In [13]:
data = pd.read_csv('normalized_data.csv')

In [10]:
from skimage.feature import hog
from PIL import Image

def hog_picture(image: Image):
    vector = hog(
        np.asarray(image),
        orientations=8,
        pixels_per_cell=(16, 16),
        cells_per_block=(1, 1),
        channel_axis=-1,
        feature_vector=True,
    )
    return vector


In [14]:
import io
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor
from PIL import Image, ImageOps
from ipywidgets import IntProgress
from IPython.display import display

@dataclass()
class Metadata:
    severity: int
    relative_path: str
    hog: np.array

def handle_file(item) -> Metadata:
    buffer = io.BytesIO()
    s3_client.download_fileobj(BUCKET_NAME, f'{DATASET_DIR}{item.relative_path}', buffer)
    buffer.seek(0)
    img = Image.open(buffer)
    hog_fd = hog_picture(ImageOps.grayscale(img))
    return Metadata(item.severity, item.relative_path, hog_fd)

raw_data = []
progress = IntProgress(min=0, max=len(data))
display(progress)
with ThreadPoolExecutor(max_workers=50) as pool:
    for item in pool.map(handle_file, data.itertuples()):
        raw_data.append(item)
        progress.value += 1


IntProgress(value=0, max=3278)

ValueError: Only images with two spatial dimensions are supported. If using with color/multichannel images, specify `channel_axis`.